In [35]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import hvplot.pandas

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import re

from datetime import date, datetime, timedelta
#display dataframes fully
pd.set_option("display.max_rows", None, "display.max_columns", None)


In [2]:
## Collect JSON file from ApeWisdom

In [3]:
top_25_mentions_crypto = requests.get('https://apewisdom.io/api/v1.0/filter/cryptocurrency').json()['results'][:25]

In [4]:
top_25_mentions_crypto

[{'rank': 1,
  'ticker': 'BTC.X',
  'name': 'Bitcoin',
  'mentions': '416',
  'upvotes': '4474',
  'rank_24h_ago': '1',
  'mentions_24h_ago': '618'},
 {'rank': 2,
  'ticker': 'LUNA.X',
  'name': 'Terra',
  'mentions': '234',
  'upvotes': '2272',
  'rank_24h_ago': '4',
  'mentions_24h_ago': '136'},
 {'rank': 3,
  'ticker': 'UST.X',
  'name': 'TerraUSD',
  'mentions': '216',
  'upvotes': '4978',
  'rank_24h_ago': '3',
  'mentions_24h_ago': '138'},
 {'rank': 4,
  'ticker': 'ETH.X',
  'name': 'Ethereum',
  'mentions': '192',
  'upvotes': '942',
  'rank_24h_ago': '2',
  'mentions_24h_ago': '409'},
 {'rank': 5,
  'ticker': 'AMA.X',
  'name': 'AMATEN',
  'mentions': '92',
  'upvotes': '1922',
  'rank_24h_ago': '51',
  'mentions_24h_ago': '5'},
 {'rank': 6,
  'ticker': 'BNB.X',
  'name': 'Binance Coin',
  'mentions': '85',
  'upvotes': '1178',
  'rank_24h_ago': '22',
  'mentions_24h_ago': '12'},
 {'rank': 7,
  'ticker': 'USDT.X',
  'name': 'Tether',
  'mentions': '73',
  'upvotes': '290',
  'r

In [5]:
## Create dataframe to work with inside pandas

In [6]:
top_25_mentions_crypto_df = pd.DataFrame.from_records(top_25_mentions_crypto)
top_25_mentions_crypto_df

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,BTC.X,Bitcoin,416,4474,1,618
1,2,LUNA.X,Terra,234,2272,4,136
2,3,UST.X,TerraUSD,216,4978,3,138
3,4,ETH.X,Ethereum,192,942,2,409
4,5,AMA.X,AMATEN,92,1922,51,5
5,6,BNB.X,Binance Coin,85,1178,22,12
6,7,USDT.X,Tether,73,290,5,104
7,8,BUSD.X,Binance USD,43,1014,25,12
8,9,USDC.X,USD Coin,41,68,8,75
9,10,ADA.X,Cardano,41,88,6,102


In [7]:
type(top_25_mentions_crypto_df['ticker'].iloc[0])

str

In [8]:
top_25_mentions_crypto_df['ticker'] = top_25_mentions_crypto_df['ticker'].str[:-2]


In [9]:
top_tickers = top_25_mentions_crypto_df.loc[:,'ticker']
top_tickers

0      BTC
1     LUNA
2      UST
3      ETH
4      AMA
5      BNB
6     USDT
7     BUSD
8     USDC
9      ADA
10     APY
11    SHIB
12     HEX
13     XRP
14     FTX
15     SOL
16    DOGE
17    ALGO
18     DOT
19     LTC
20     BCH
21     CRO
22     FTM
23     CDC
24     DAO
Name: ticker, dtype: object

In [10]:
# https://www.adamsmith.haus/python/answers/how-to-convert-a-column-in-a-pandas-dataframe-to-a-numpy-array-in-python#:~:text=Use%20pandas.,Use%20pandas.
# create variable to call for values only ** was getting error message because, for some reason-although this error was not present in the social meme file, it was not allowing to put data in a list using a for loop.

In [11]:
top_tickers_crypto = top_tickers.values
top_tickers_crypto

array(['BTC', 'LUNA', 'UST', 'ETH', 'AMA', 'BNB', 'USDT', 'BUSD', 'USDC',
       'ADA', 'APY', 'SHIB', 'HEX', 'XRP', 'FTX', 'SOL', 'DOGE', 'ALGO',
       'DOT', 'LTC', 'BCH', 'CRO', 'FTM', 'CDC', 'DAO'], dtype=object)

In [12]:
top_25_mentions_crypto_df.set_index('ticker', inplace=True)
top_25_mentions_crypto_df

,rank,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
ticker,,,,,,
BTC,1,Bitcoin,416,4474,1,618
LUNA,2,Terra,234,2272,4,136
UST,3,TerraUSD,216,4978,3,138
ETH,4,Ethereum,192,942,2,409
AMA,5,AMATEN,92,1922,51,5
BNB,6,Binance Coin,85,1178,22,12
USDT,7,Tether,73,290,5,104
BUSD,8,Binance USD,43,1014,25,12
USDC,9,USD Coin,41,68,8,75


In [13]:
top_25_mentions_crypto_df = pd.DataFrame.from_dict(top_25_mentions_crypto_df, orient='columns')
top_25_mentions_crypto_df

,rank,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
ticker,,,,,,
BTC,1,Bitcoin,416,4474,1,618
LUNA,2,Terra,234,2272,4,136
UST,3,TerraUSD,216,4978,3,138
ETH,4,Ethereum,192,942,2,409
AMA,5,AMATEN,92,1922,51,5
BNB,6,Binance Coin,85,1178,22,12
USDT,7,Tether,73,290,5,104
BUSD,8,Binance USD,43,1014,25,12
USDC,9,USD Coin,41,68,8,75


In [14]:
top_25_mentions_crypto_df.drop(columns=['rank','name','upvotes','rank_24h_ago','mentions_24h_ago'],inplace=True)                              
top_25_mentions_crypto_df

,mentions
ticker,
BTC,416
LUNA,234
UST,216
ETH,192
AMA,92
BNB,85
USDT,73
BUSD,43
USDC,41


In [15]:
# type(top_10_mentions_crypto_df['mentions'].iloc[0])

In [16]:
top_25_mentions_crypto_df = pd.to_numeric(top_25_mentions_crypto_df['mentions'], errors='coerce')

In [17]:
top_25_mentions_crypto_df

ticker
BTC     416
LUNA    234
UST     216
ETH     192
AMA      92
BNB      85
USDT     73
BUSD     43
USDC     41
ADA      41
APY      40
SHIB     39
HEX      35
XRP      35
FTX      32
SOL      29
DOGE     25
ALGO     24
DOT      23
LTC      22
BCH      21
CRO      20
FTM      19
CDC      18
DAO      18
Name: mentions, dtype: int64

In [18]:
top_25_mentions_crypto_plot = top_25_mentions_crypto_df.hvplot(kind="bar", rot = 90, title='Mentions in last 24 hours for top 25 cryptocurrencies')
top_25_mentions_crypto_plot

:Bars   [ticker]   (mentions)

In [19]:
# modifies ticker string to make call compatible with alpaca.

modified_ticker_alpaca = []

for crypto in top_25_mentions_crypto_df.index:
    modified_ticker_alpaca.append(crypto+'USD')
modified_ticker_alpaca

['BTCUSD',
 'LUNAUSD',
 'USTUSD',
 'ETHUSD',
 'AMAUSD',
 'BNBUSD',
 'USDTUSD',
 'BUSDUSD',
 'USDCUSD',
 'ADAUSD',
 'APYUSD',
 'SHIBUSD',
 'HEXUSD',
 'XRPUSD',
 'FTXUSD',
 'SOLUSD',
 'DOGEUSD',
 'ALGOUSD',
 'DOTUSD',
 'LTCUSD',
 'BCHUSD',
 'CROUSD',
 'FTMUSD',
 'CDCUSD',
 'DAOUSD']

In [20]:
#modified ticker string structure

print(modified_ticker_alpaca)

['BTCUSD', 'LUNAUSD', 'USTUSD', 'ETHUSD', 'AMAUSD', 'BNBUSD', 'USDTUSD', 'BUSDUSD', 'USDCUSD', 'ADAUSD', 'APYUSD', 'SHIBUSD', 'HEXUSD', 'XRPUSD', 'FTXUSD', 'SOLUSD', 'DOGEUSD', 'ALGOUSD', 'DOTUSD', 'LTCUSD', 'BCHUSD', 'CROUSD', 'FTMUSD', 'CDCUSD', 'DAOUSD']


In [21]:
load_dotenv('.env')

True

In [22]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
type(alpaca_api_key)

str

In [23]:
# Returns the current local date
todays_date = date.today()
seven_days_back_date = todays_date - timedelta(7)

alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = 'v2')
start_date = pd.Timestamp(seven_days_back_date, tz='America/New_York').isoformat()
end_date = pd.Timestamp(todays_date, tz='America/New_York').isoformat() 

prices_df = alpaca.get_crypto_bars(modified_ticker_alpaca[0:15], timeframe = '24Hr', start = seven_days_back_date).df
prices_df.dropna()
prices_df.tail(5)

,exchange,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,,
2022-05-18 00:00:00+00:00,FTXU,0.9988,0.9991,0.9985,0.9988,908192.38,314,0.998685,USDTUSD
2022-05-19 00:00:00+00:00,FTXU,0.9988,0.9990,0.9984,0.9990,1981680.83,340,0.998892,USDTUSD
2022-05-20 00:00:00+00:00,FTXU,0.9989,0.9993,0.9916,0.9988,2234293.13,358,0.998482,USDTUSD
2022-05-21 00:00:00+00:00,FTXU,0.9987,0.9988,0.9985,0.9987,211535.41,225,0.998681,USDTUSD
2022-05-22 00:00:00+00:00,FTXU,0.9986,0.9987,0.9986,0.9987,57326.67,43,0.998602,USDTUSD


In [24]:
crypto_compatible_prices = prices_df.symbol.unique()
crypto_compatible_prices

array(['BTCUSD', 'ETHUSD', 'SHIBUSD', 'USDTUSD'], dtype=object)

In [25]:
prices_pivot_df = (prices_df.pivot_table(columns='symbol', values='close', index='timestamp').pct_change()+1).cumprod()
    

In [26]:
prices_df_plot = prices_pivot_df.hvplot(kind='line', rot=90, title='Crypto price volatility')
prices_df_plot

:NdOverlay   [symbol]
   :Curve   [timestamp]   (value)

In [65]:
# top_5_mentions_crypto_list = [modified_ticker[0:3] for modified_ticker in prices_pivot_df.columns]
top_5_mentions_crypto_list = [modified_ticker.replace(modified_ticker[len(modified_ticker) - len('USD') : ], '') for modified_ticker in prices_pivot_df.columns]
top_5_mentions_crypto_list

# text = text.replace(text[size - 3:], replacement)

['BTC', 'ETH', 'SHIB', 'T']

In [46]:
top_25_mentions_crypto_df

ticker
BTC     416
LUNA    234
UST     216
ETH     192
AMA      92
BNB      85
USDT     73
BUSD     43
USDC     41
ADA      41
APY      40
SHIB     39
HEX      35
XRP      35
FTX      32
SOL      29
DOGE     25
ALGO     24
DOT      23
LTC      22
BCH      21
CRO      20
FTM      19
CDC      18
DAO      18
Name: mentions, dtype: int64

In [32]:
top_5_mentions_crypto_df = top_25_mentions_crypto_df.filter(items = top_5_mentions_crypto_list, axis=0)
top_5_mentions_crypto_df

ticker
BTC    416
ETH    192
Name: mentions, dtype: int64

In [30]:
top_5_mentions_crypto_plot = top_5_mentions_crypto_df.hvplot(kind="bar", rot = 90, title='Mentions in last 24 hours for top 5 cryptocurrencies')
top_5_mentions_crypto_plot

:Bars   [ticker]   (mentions)

In [28]:
overlayplot = prices_df_plot + top_5_mentions_crypto_plot


In [29]:
overlayplot

:Layout
   .NdOverlay.I   :NdOverlay   [symbol]
      :Curve   [timestamp]   (value)
   .Bars.Mentions :Bars   [ticker]   (mentions)